In [1]:
import pandas as pd
import os

In [8]:
num_files = len(os.listdir('spread_probability'))

In [ ]:
df = pd.read_csv('spread_probability/1.csv')
df

In [2]:
def cal_compress(from_df, geoid, geoid_level):
    from_df = from_df.groupby('to_%s'%geoid_level)['prob'].sum().reset_index()
    from_df['from_%s'%geoid_level] = len(from_df) * [geoid]
    from_df['prob'] = from_df['prob']/from_df['prob'].sum()
    from_df = from_df[['from_%s'%geoid_level,'to_%s'%geoid_level,'prob']]
    return from_df

In [4]:
df_list = []
for i in range(5):
    tmp_df = pd.read_csv('spread_probability_top30/cbg/%d.csv'%(i+1))
    tmp_df['from'] = tmp_df['from'].apply(lambda x: '%012d'%x)
    tmp_df['to'] = tmp_df['to'].apply(lambda x: '%012d'%x)
    tmp_df['from_county'] = tmp_df['from'].apply(lambda x: x[:5])
    tmp_df['from_tract'] = tmp_df['from'].apply(lambda x: x[:11])
    tmp_df['to_county'] = tmp_df['to'].apply(lambda x: x[:5])
    tmp_df['to_tract'] = tmp_df['to'].apply(lambda x: x[:11])
    
    df_list.append(tmp_df)
    
df_full = pd.concat(df_list)
df_county_list = []
from_county = df_full['from_county'].unique().tolist()
df_county_group = df_full.groupby('from_county')
print(f'Total {len(from_county)} Counties. ')
for ci,county in enumerate(from_county):
    df_county_list.append(cal_compress(df_county_group.get_group(county), county, 'county'))
    print(f'  {ci+1}/{len(from_county)}')
df_county = pd.concat(df_county_list)
df_county.to_csv('spread_probability_top30/county.csv', index = False)

df_tract_list = []
fi = 0
from_tract = df_full['from_tract'].unique().tolist()
df_tract_group = df_full.groupby('from_tract')
print(f'Total {len(from_tract)} Census Tracts. ')
for ti, tract in enumerate(from_tract):
    df_tract_list.append(cal_compress(df_tract_group.get_group(tract), tract, 'tract'))
    if (ti+1)% 1000 == 0: print(f' {ti+1}/{len(from_tract)}')
df_tract = pd.concat(df_tract_list)
df_tract.to_csv('spread_probability_top30/tract.csv', index = False)
    
    

Total 3142 Counties. 
  1/3142
  2/3142
  3/3142
  4/3142
  5/3142
  6/3142
  7/3142
  8/3142
  9/3142
  10/3142
  11/3142
  12/3142
  13/3142
  14/3142
  15/3142
  16/3142
  17/3142
  18/3142
  19/3142
  20/3142
  21/3142
  22/3142
  23/3142
  24/3142
  25/3142
  26/3142
  27/3142
  28/3142
  29/3142
  30/3142
  31/3142
  32/3142
  33/3142
  34/3142
  35/3142
  36/3142
  37/3142
  38/3142
  39/3142
  40/3142
  41/3142
  42/3142
  43/3142
  44/3142
  45/3142
  46/3142
  47/3142
  48/3142
  49/3142
  50/3142
  51/3142
  52/3142
  53/3142
  54/3142
  55/3142
  56/3142
  57/3142
  58/3142
  59/3142
  60/3142
  61/3142
  62/3142
  63/3142
  64/3142
  65/3142
  66/3142
  67/3142
  68/3142
  69/3142
  70/3142
  71/3142
  72/3142
  73/3142
  74/3142
  75/3142
  76/3142
  77/3142
  78/3142
  79/3142
  80/3142
  81/3142
  82/3142
  83/3142
  84/3142
  85/3142
  86/3142
  87/3142
  88/3142
  89/3142
  90/3142
  91/3142
  92/3142
  93/3142
  94/3142
  95/3142
  96/3142
  97/3142
  98/3142
  99/31

In [ ]:
src_cbgs = df['from'].unique().tolist()
grouped_src = df.groupby('from')
src_cbg_df = grouped_src.get_group(src_cbgs[0]).sort_values(by='prob', ascending=False).reset_index(drop=True)
src_cbg_df = src_cbg_df.iloc[:30]
sum_prob = src_cbg_df['prob'].sum()
src_cbg_df['prob'] = src_cbg_df['prob']/sum_prob
src_cbg_df

In [ ]:
def cut_each_src(src_cbg_df):
    src_cbg_df = src_cbg_df.sort_values(by='prob', ascending=False).reset_index(drop=True)
    src_cbg_df = src_cbg_df.iloc[:30]
    sum_prob = src_cbg_df['prob'].sum()
    src_cbg_df['prob'] = src_cbg_df['prob']/sum_prob
    return src_cbg_df

In [ ]:
os.mkdir('spread_probability_top30')

In [ ]:
res = []

for i in range(num_files):
    df = pd.read_csv('spread_probability/%d.csv'%(i+1))
    src_cbgs = df['from'].unique().tolist()
    grouped_src = df.groupby('from')

    for src_cbg in src_cbgs:
        res.append(cut_each_src(grouped_src.get_group(src_cbg)))

    print(i, 'done')

    if (i+1) %50 == 0:
        pd.concat(res).to_csv('spread_probability_top30/%d.csv'%((i+1)//50), index=False)
        res = []
        
pd.concat(res).to_csv('spread_probability_top30/{}.csv'.format(num_files// 50+1), index=False)

In [ ]:
df = pd.read_csv('spread_probability_top30/cbg/1.csv')
df

In [ ]:
df['from'] = df['from'].apply(lambda x: int(('%012d'%x)[:5]))
df['to'] = df['to'].apply(lambda x: int(('%012d'%x)[:5]))
df

In [ ]:
from_county = df['from'].unique().tolist()
grouped_from_county = df.groupby('from')
from_df = grouped_from_county.get_group(from_county[0])
from_df

In [ ]:
from_df['prob'].sum()

In [ ]:
from_c = from_df['from'].tolist()[0]
from_df = from_df.groupby('to')['prob'].sum().reset_index()
from_df['from'] = len(from_df) * [from_c]
from_df['prob'] = from_df['prob']/from_df['prob'].sum()
from_df = from_df[['from','to','prob']]

In [ ]:
def cal_county(from_df, county):
    from_df = from_df.groupby('to')['prob'].sum().reset_index()
    from_df['from'] = len(from_df) * [county]
    from_df['prob'] = from_df['prob']/from_df['prob'].sum()
    from_df = from_df[['from','to','prob']]
    return from_df

In [ ]:
res = []
for i in range(1,5):
    df = pd.read_csv('spread_probability_top30/%d.csv'%i)
    df['from'] = df['from'].apply(lambda x: int(('%012d'%x)[:5]))
    df['to'] = df['to'].apply(lambda x: int(('%012d'%x)[:5]))
    from_county = df['from'].unique().tolist()
    grouped_from_county = df.groupby('from')

    for county in from_county:
        res.append(cal_county(grouped_from_county.get_group(county),county))
res = pd.concat(res)
res

In [ ]:
res.to_csv('spread_probability_top30/county_level.csv', index=False)